In [ ]:
using HierarchicalEOM
using CairoMakie
using Printf

# Paramètres du système
ϵ = -5.0
U = 10.0
σm = sigmam()
σz = sigmaz()
II = qeye(2)

# Opérateurs d'annihilation (transformation de Jordan-Wigner)
d_up = tensor(σm, II)
d_dn = tensor(-1 * σz, σm)
Hsys = ϵ * (d_up' * d_up + d_dn' * d_dn) + U * (d_up' * d_up * d_dn' * d_dn)

# Paramètres du réservoir
μ = 0.0      # Potentiel chimique
W = 10.0     # Largeur de bande
kT = 0.5     # Température
N = 5        # Nombre de termes exponentiels
tier = 3     # Niveau de hiérarchie

# Liste des couplages pour régime faible et fort
Γ_weak = [0.5, 1.0, 2.0, 5.0, 10.0]    # U/Γ = 20, 10, 5, 2, 1
Γ_strong = [0.01, 0.02, 0.05, 0.1, 0.2]  # U/Γ = 1000, 500, 200, 100, 50

# États de base
rho_0 = tensor(basis(2, 0), basis(2, 0))
rho_0 = rho_0 * rho_0'
rho_1 = tensor(basis(2, 0), basis(2, 1))
rho_1 = rho_1 * rho_1'
rho_2 = tensor(basis(2, 1), basis(2, 0))
rho_2 = rho_2 * rho_2'
rho_3 = tensor(basis(2, 1), basis(2, 1))
rho_3 = rho_3 * rho_3'

# Temps d'évolution
tlist = 0:0.2:50

# Fonction pour calculer l'évolution temporelle
function compute_evolution(Γ, tlist)
    println("Calcul pour Γ = $Γ (U/Γ = $(U/Γ))")
    
    # Bains fermioniques
    bath_up = Fermion_Lorentz_Pade(d_up, Γ, μ, W, kT, N)
    bath_dn = Fermion_Lorentz_Pade(d_dn, Γ, μ, W, kT, N)
    bath_list = [bath_up, bath_dn]
    
    # Matrice HEOM
    M_even = M_Fermion(Hsys, tier, bath_list)
    
    # Évolution temporelle
    sol = HEOMsolve(M_even, rho_0, tlist, e_ops=[rho_0, rho_1, rho_2, rho_3])
    
    # Extraction des populations
    p11 = real(sol.expect[1, :])
    p22 = real(sol.expect[2, :])
    p33 = real(sol.expect[3, :])
    p44 = real(sol.expect[4, :])
    
    return p11, p22, p33, p44
end

# Fonction pour créer une figure avec sous-graphiques
function plot_populations(Γ_list, regime_name)
    n_plots = length(Γ_list)
    
    # Créer la figure avec grille de sous-graphiques
    fig = Figure(size=(1200, 800))
    
    # Titre principal
    Label(fig[0, :], text="Évolution des populations - Régime $regime_name", 
          fontsize=20, font=:bold)
    
    # Créer les sous-graphiques
    for (i, Γ) in enumerate(Γ_list)
        row = div(i-1, 3) + 1
        col = mod(i-1, 3) + 1
        
        # Calculer l'évolution
        p11, p22, p33, p44 = compute_evolution(Γ, tlist)
        
        # Créer l'axe
        ax = Axis(fig[row, col],
                  xlabel="t",
                  ylabel="ρ(t)",
                  title=@sprintf("Γ = %.2f (U/Γ = %.1f)", Γ, U/Γ))
        
        # Tracer les courbes
        lines!(ax, tlist, p11, color=:blue, label="ρ₁₁", linewidth=2)
        lines!(ax, tlist, p22, color=:red, label="ρ₂₂", linewidth=2)
        lines!(ax, tlist, p33, color=:green, label="ρ₃₃", linewidth=2)
        lines!(ax, tlist, p44, color=:purple, label="ρ₄₄", linewidth=2)
        
        # Ajouter la légende
        if i == 1
            Legend(fig[row, col+1], ax, framevisible=true)
        end
    end
    
    return fig
end

# Générer les graphiques pour les deux régimes
println("\n=== RÉGIME FAIBLE COUPLAGE ===")
fig_weak = plot_populations(Γ_weak, "Faible Couplage")
display(fig_weak)
# save("populations_weak_coupling.png", fig_weak)

println("\n=== RÉGIME FORT COUPLAGE ===")
fig_strong = plot_populations(Γ_strong, "Fort Couplage")
display(fig_strong)
# save("populations_strong_coupling.png", fig_strong)

# Fonction alternative : tous les graphiques sur une seule figure
function plot_all_populations_single(Γ_list, regime_name)
    fig = Figure(size=(1000, 600))
    
    ax = Axis(fig[1, 1],
              xlabel="t",
              ylabel="ρ(t)",
              title="Évolution des populations - Régime $regime_name")
    
    colors = [:blue, :red, :green, :purple, :orange]
    
    for (i, Γ) in enumerate(Γ_list)
        println("Calcul pour Γ = $Γ")
        p11, p22, p33, p44 = compute_evolution(Γ, tlist)
        
        # Tracer uniquement ρ₃₃ (état doublement occupé) pour comparaison
        lines!(ax, tlist, p33, 
               color=colors[i], 
               label=@sprintf("Γ=%.2f (U/Γ=%.0f)", Γ, U/Γ),
               linewidth=2)
    end
    
    Legend(fig[1, 2], ax, "U/Γ ratio", framevisible=true)
    
    return fig
end

# Comparaison sur une seule figure (population ρ₃₃)
println("\n=== COMPARAISON RÉGIME FAIBLE ===")
fig_comp_weak = plot_all_populations_single(Γ_weak, "Faible Couplage - ρ₃₃")
display(fig_comp_weak)

println("\n=== COMPARAISON RÉGIME FORT ===")
fig_comp_strong = plot_all_populations_single(Γ_strong, "Fort Couplage - ρ₃₃")
display(fig_comp_strong)